In [214]:
from lib.transformdata import CreateTableInSQLServer
from lib.transformdata import MaskData
from lib.transformdata import TransformData
import numpy as np
import random
import pandas as pd
import re
file_location = r'C:\Users\tomda\OneDrive\Desktop\Python Supper App\DataSF.xlsx'
def keep_first_number(s):
    match = re.match(r'^\s*(\d+\s+\d+)', s)
    return match.group(1) + " Campaign " if match else "Campaign "
df_Account = TransformData(file_location, 'Account').result() #Preparedata to import
df_User = TransformData(file_location, 'User').result() #Preparedata to import
df_Campaign = TransformData(file_location, 'Campaign').result()
df_Lead = TransformData(file_location, 'Lead').result() #Preparedata to import
df_Contact = TransformData(file_location, 'Contact').result() #Preparedata to import
df_CampaignMember = TransformData(file_location, 'CampaignMember').result() #Preparedata to import
df_OppProduct = TransformData(file_location, 'OppProduct').result() #Preparedata to import
df_Opportunity= TransformData(file_location, 'Opportunity').result() #Preparedata to import
df_Task= TransformData(file_location, 'Task').result() #Preparedata to import

Amount Columns: []
Date Columns: ['CreatedDate']
Amount Columns: []
Date Columns: []
Amount Columns: ['ExpectedRevenue', 'BudgetedCost', 'ActualCost', 'AmountAllOpportunities', 'AmountWonOpportunities']
Date Columns: ['StartDate', 'EndDate']
Amount Columns: ['AnnualRevenue']
Date Columns: ['CreatedDate']
Amount Columns: []
Date Columns: []
Amount Columns: []
Date Columns: ['CreatedDate']
Amount Columns: []
Date Columns: []
Amount Columns: ['Amount']
Date Columns: ['CloseDate', 'CreatedDate']
Amount Columns: []
Date Columns: ['CreatedDate']


In [215]:
# Account
df_Account = MaskData(df_Account, ['Name'], 'Full Name', r'lib\Maskdata\names.csv').mask_data() #Mask Account Name
df_Account = MaskData(df_Account, ['Phone'], 'Full Name', r'lib\Maskdata\names.csv').mask_phone() #Mask Account Phone
df_Account["Account_Id_masked"] = range(1,len(df_Account) + 1) #Mask AccountId
# User
df_User = MaskData(df_User, ['Name'], 'Full Name', r'lib\Maskdata\names.csv').mask_data() #Mask User Name
df_User["User_Id_masked"] = range(1,len(df_User) + 1) #Mask AccountId
# Campaign
df_Campaign["Campaign_Id_masked"] = range(1,len(df_Campaign) + 1) #Mask CampaignId
df_Campaign["Name"] = df_Campaign["Name"].apply(keep_first_number)  + df_Campaign["Campaign_Id_masked"].astype(str)
df_Campaign['Type'] = df_Campaign['Type'].str.replace('CF', 'SS') #Replace CF to SS in Type
    # Replace OwnerId by new userId
merged_df = pd.merge(df_Campaign, df_User[['Id', 'User_Id_masked']], left_on='OwnerId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['OwnerId'] = merged_df['User_Id_masked']
df_Campaign = merged_df[df_Campaign.columns]
# Contact
df_Contact["Contact_Id_masked"] = range(1,len(df_Contact) + 1) #Mask ContactId
    # replace AccountId by Account_Id_masked
merged_df = pd.merge(df_Contact, df_Account[['Id', 'Account_Id_masked']], left_on='AccountId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['AccountId'] = merged_df['Account_Id_masked']
df_Contact = merged_df[df_Contact.columns]
# Lead
df_Lead = MaskData(df_Lead, ['Name'], 'Full Name', r'lib\Maskdata\names.csv').mask_data() #Mask Lead Name
df_Lead["Lead_Id_masked"] = range(1,len(df_Lead) + 1) #Mask LeadId
    #Create name of company by group and increament 
temp_df = pd.DataFrame(df_Lead['Company'].unique(), columns=['Company']) 
temp_df['temp_id'] = range(1, len(temp_df) + 1)
temp_df['temp_CMP'] = "Company " +  temp_df['temp_id'].astype(str)
merged_df = pd.merge(df_Lead, temp_df[['Company', 'temp_CMP']], left_on='Company', right_on='Company', how='left', suffixes=('', '_y'))
merged_df['Company'] = merged_df['temp_CMP'] # Mask Company
df_Lead = merged_df[df_Lead.columns]
# CampaignMember
df_CampaignMember["CampaignMember_Id_masked"] = range(1,len(df_CampaignMember) + 1) #Mask CampaignMember Id
    # replace LeadId by Lead_Id_masked
merged_df = pd.merge(df_CampaignMember, df_Lead[['Id', 'Lead_Id_masked']], left_on='LeadId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['LeadId'] = merged_df['Lead_Id_masked']
df_CampaignMember = merged_df[df_CampaignMember.columns]
    # replace ContactId by Contact_Id_masked
merged_df = pd.merge(df_CampaignMember, df_Contact[['Id', 'Contact_Id_masked']], left_on='ContactId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['ContactId'] = merged_df['Contact_Id_masked']
df_CampaignMember = merged_df[df_CampaignMember.columns]
# Opportunity
df_Opportunity["Opportunity_Id_masked"] = range(1,len(df_Opportunity) + 1) #Mask Opportunity Id
    # replace AccountId by Account_Id_masked
merged_df = pd.merge(df_Opportunity, df_Account[['Id', 'Account_Id_masked']], left_on='AccountId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['AccountId'] = merged_df['Account_Id_masked']
df_Opportunity = merged_df[df_Opportunity.columns]
    # replace OwnerId by User_Id_masked
merged_df = pd.merge(df_Opportunity, df_User[['Id', 'User_Id_masked']], left_on='OwnerId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['OwnerId'] = merged_df['User_Id_masked']
df_Opportunity = merged_df[df_Opportunity.columns]
    # replace ContactId by Contact_Id_masked
merged_df = pd.merge(df_Opportunity, df_Contact[['Id', 'Contact_Id_masked']], left_on='ContactId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['ContactId'] = merged_df['Contact_Id_masked']
df_Opportunity = merged_df[df_Opportunity.columns]
    # replace CampaignId by Campaign_Id_masked
merged_df = pd.merge(df_Opportunity, df_Campaign[['Id', 'Campaign_Id_masked']], left_on='CampaignId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['CampaignId'] = merged_df['Campaign_Id_masked']
df_Opportunity = merged_df[df_Opportunity.columns]
df_Opportunity['LeadSource'] = df_Opportunity['LeadSource'].str.replace('CF', 'SS') #Replace CF to SS in LeadSource
# OppProduct
df_OppProduct["OppProduct_Id_masked"] = range(1,len(df_OppProduct) + 1) #Mask Product Id
    # replace OpportunityId by Opportunity_Id_masked
merged_df = pd.merge(df_OppProduct, df_Opportunity[['Id', 'Opportunity_Id_masked']], left_on='OpportunityId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['OpportunityId'] = merged_df['Opportunity_Id_masked']
df_OppProduct = merged_df[df_OppProduct.columns]
# Task
df_Task["Task_Id_masked"] = range(1,len(df_Task) + 1) #Mask Task Id
df_Task = MaskData(df_Task, ['Name'], 'Full Name', r'lib\Maskdata\names.csv').mask_data() #Mask Task Name
df_Task['Description'] = "Description " + df_Task["Task_Id_masked"].astype(str)
    # replace OwnerId by User_Id_masked
merged_df = pd.merge(df_Task, df_User[['Id', 'User_Id_masked']], left_on='OwnerId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['OwnerId'] = merged_df['User_Id_masked']
df_Task = merged_df[df_Task.columns]
    # replace AccountId by Account_Id_masked
merged_df = pd.merge(df_Task, df_Account[['Id', 'Account_Id_masked']], left_on='AccountId', right_on='Id', how='left',suffixes=('', '_y'))
merged_df['AccountId'] = merged_df['Account_Id_masked']
df_Task = merged_df[df_Task.columns]

C:\Users\tomda\AppData\Local\Temp\ipykernel_5156\3245973703.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Opportunity['LeadSource'] = df_Opportunity['LeadSource'].str.replace('CF', 'SS') #Replace CF to SS in LeadSource


In [216]:
df_Account['Id'] = df_Account['Account_Id_masked'].astype(int)
df_Account.drop(columns=['Account_Id_masked'], inplace=True)
df_User['Id'] = df_User['User_Id_masked'].astype(int)
df_User.drop(columns=['User_Id_masked'], inplace=True)
df_Campaign['Id'] = df_Campaign['Campaign_Id_masked'].astype(int)
df_Campaign.drop(columns=['Campaign_Id_masked'], inplace=True)
df_Contact['Id'] = df_Contact['Contact_Id_masked'].astype(int)
df_Contact.drop(columns=['Contact_Id_masked'], inplace=True)
df_Lead['Id'] = df_Lead['Lead_Id_masked'].astype(int)
df_Lead.drop(columns=['Lead_Id_masked'], inplace=True)
df_CampaignMember['Id'] = df_CampaignMember['CampaignMember_Id_masked'].astype(int)
df_CampaignMember.drop(columns=['CampaignMember_Id_masked'], inplace=True)
df_CampaignMember['LeadId'].astype('Int64')
df_CampaignMember['ContactId'].astype('Int64')
df_Opportunity['Id'] = df_Opportunity['Opportunity_Id_masked'].astype(int)
df_Opportunity.drop(columns=['Opportunity_Id_masked'], inplace=True)
df_Opportunity['AccountId'].astype('Int64')
df_Opportunity['ContactId'].astype('Int64')
df_Opportunity['CampaignId'].astype('Int64')
df_OppProduct['Id'] = df_OppProduct['OppProduct_Id_masked'].astype(int)
df_OppProduct.drop(columns=['OppProduct_Id_masked'], inplace=True)
df_OppProduct['OpportunityId'].astype('Int64')
df_Task['Id'] = df_Task['Task_Id_masked'].astype(int)
df_Task.drop(columns=['Task_Id_masked'], inplace=True)
df_Task['OwnerId'].astype('Int64')
df_Task['AccountId'].astype('Int64')


C:\Users\tomda\AppData\Local\Temp\ipykernel_5156\764941819.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Task['Id'] = df_Task['Task_Id_masked'].astype(int)
C:\Users\tomda\AppData\Local\Temp\ipykernel_5156\764941819.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Task.drop(columns=['Task_Id_masked'], inplace=True)


0         562
1         563
2         537
3         635
4         704
         ... 
34487    5166
34488    <NA>
34489    3018
34490    6785
34491    <NA>
Name: AccountId, Length: 34492, dtype: Int64

In [217]:
CreateTableInSQLServer('TOM', 'SampleSF', 'Account', df_Account).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'CRM_User', df_User).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'Contact', df_Contact).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'Campaign', df_Campaign).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'CampaignMember', df_CampaignMember).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'Opportunity', df_Opportunity).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'OppProduct', df_OppProduct).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'Task', df_Task).run()
CreateTableInSQLServer('TOM', 'SampleSF', 'Lead', df_Lead).run()

OK
OK
OK
OK
OK
OK
OK
OK
OK
